# 빅데이터분석(21_2)
### 휴먼지능정보공학과 201810793 이연우
* 3주차 과제
---

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#휴먼지능정보공학과-201810793-이연우" data-toc-modified-id="휴먼지능정보공학과-201810793-이연우-0.1">휴먼지능정보공학과 201810793 이연우</a></span></li></ul></li><li><span><a href="#문제-1" data-toc-modified-id="문제-1-1">문제 1</a></span></li><li><span><a href="#문제-2" data-toc-modified-id="문제-2-2">문제 2</a></span></li><li><span><a href="#문제-3:" data-toc-modified-id="문제-3:-3">문제 3:</a></span><ul class="toc-item"><li><span><a href="#1)-경기도-의정부시-인구현황-RDD-읽기" data-toc-modified-id="1)-경기도-의정부시-인구현황-RDD-읽기-3.1">1) 경기도 의정부시 인구현황 RDD 읽기</a></span></li><li><span><a href="#2)-제주특별자치도-서귀포시-내-연도별-65세이상-인구수-및-고령화비율,-노령화지수-현황-RDD-읽기" data-toc-modified-id="2)-제주특별자치도-서귀포시-내-연도별-65세이상-인구수-및-고령화비율,-노령화지수-현황-RDD-읽기-3.2">2) 제주특별자치도 서귀포시 내 연도별 65세이상 인구수 및 고령화비율, 노령화지수 현황 RDD 읽기</a></span></li><li><span><a href="#3)-한글-깨짐-해결-방법-(binaryFile)" data-toc-modified-id="3)-한글-깨짐-해결-방법-(binaryFile)-3.3">3) 한글 깨짐 해결 방법 (binaryFile)</a></span><ul class="toc-item"><li><span><a href="#4.-데이터-분리하기-(DataFrame)" data-toc-modified-id="4.-데이터-분리하기-(DataFrame)-3.3.1">4. 데이터 분리하기 (DataFrame)</a></span></li></ul></li></ul></li></ul></div>

---
## 문제 1

오픈API를 통해 데이터를 5천 ~ 1만 건 정도 수집해서 파일로 저장한다 (초과해도 좋다).

열린데이터, 공공데이터 또는 (할 수 있다면) 트위터와 같은 SNS, 댓글 등 적합한 소스를 자신이 선택한다.

과정의 후반으로 가서, 이 데이터를 분석하는 과제가 주어지게 된다.

프로그램에서 몇 건을 읽었는지와 마지막 줄을 화면 출력한다.

!dir 명령어로 파일이 존재하는지 출력한다. 1만 건 정도 저장되었으니 파일의 크기가 적당한지 가늠해보자

- 서울시 코로나19 확진자 현황 
Open_API : http://data.seoul.go.kr/dataList/OA-20279/S/1/datasetView.do

In [1]:
%%writefile src/ds_Corona19Status.py
# coding: utf-8
import os
import requests
import json
from pymongo import MongoClient
import mylib

Client = MongoClient('localhost:27017') # 클라이언트 만들기
_db=Client['ds_Corona19StatusDb'] #db created by mongo. You do not have to create this. db명 
_table=_db['ds_Corona19StatusTable'] #collection table 명
#db=Client.ds_rest_subwayPassengers

def saveJson(_fname,_data):
    import io
    with io.open(_fname, 'a', encoding='utf8') as json_file: # a(append) = 붙여넣기 추가
        #_j=json.dumps(_data, json_file, ensure_ascii=False, encoding='utf8') # python2
        _j=json.dump(_data, json_file, ensure_ascii=False)
        json_file.write(str(_j)+"\n")

def readJson(_fname):
    for line in open(_fname, 'r').readlines():
        _j=json.loads(line)
        #print _j['id'],_j['text']
        #print (_j['id'])

def saveDB(_data):
    _table.insert_one(_data)

def readDB():
    for tweet in _table.find():
        print (tweet['id'],tweet['text'])

def saveFile(_fname,_data):
    fp=open(_fname,'a')
    fp.write(_data+"\n")

def doIt():
    keyPath=os.path.join(os.getcwd(), 'src', 'key.properties')
    key=mylib.getKey(keyPath)  

    _url='http://openAPI.seoul.go.kr:8088'
    _key=str(key['03_HW'])
    _type='json'
    _service='Corona19Status'
    _start_index=1
    _end_index=1
    
    _maxIter=20
    _iter=0
    _jfname='src/ds_Corona19Status.json'
    
    while _iter<_maxIter:
        _api="/".join([_url,_key,_type,_service,str(_start_index),str(_end_index)])
        #print _api
        r=requests.get(_api)
        _json=r.json()
        #print (_json)
        # 저장 
        saveJson(_jfname,_json)
        saveDB(_json)
        # /저장
        _start_index+=1
        _end_index+=1
        _iter+=1
        if _iter >= _maxIter:
            print(_json)

if __name__ == "__main__":
    doIt()

Overwriting src/ds_Corona19Status.py


- 프로그램에서 몇 건을 읽었는지와 마지막 줄

total : 90532개

In [2]:
!python src/ds_Corona19Status.py

{'Corona19Status': {'list_total_count': 90532, 'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'}, 'row': [{'CORONA19_ID': '90513', 'CORONA19_DATE': '2021-09-17', 'CORONA19_NO': '', 'CORONA19_COUNTRY': '', 'CORONA19_PERSONAL': '', 'CORONA19_AREA': '강동구', 'CORONA19_TRAVEL_HISTORY': '', 'CORONA19_CONTACT_HISTORY': '감염경로 조사중', 'CORONA19_CORRECTIVE': '', 'CORONA19_LEAVE_STATUS': '-', 'CORONA19_MOVING_PATH': '', 'CORONA19_IDATE': '2021-09-18 10:54:18', 'CORONA19_MDATE': '2021-09-18 10:54:18'}]}, '_id': ObjectId('6145ff9d51d3f10355de7e60')}


- 파일 크기

2021-09-18  오후 11:56            11,072 ds_Corona19Status.json

In [3]:
!dir -l src\ds_Corona19Status*

 C 드라이브의 볼륨에는 이름이 없습니다.

파일을 찾을 수 없습니다.


 볼륨 일련 번호: 4E88-AB80

 C:\Users\Nunu\BigData 디렉터리


 C:\Users\Nunu\BigData\src 디렉터리

2021-09-19  오전 12:02            22,144 ds_Corona19Status.json
2021-09-19  오전 12:02             1,918 ds_Corona19Status.py
2021-09-18  오후 11:35               147 ds_Corona19Status_mongo.js
               3개 파일              24,209 바이트
               0개 디렉터리  29,289,340,928 바이트 남음


---
## 문제 2
노트북에서 pyspark 실행

- pyspark 버전

In [2]:
import pyspark
print(pyspark.__version__)

3.0.3


- java 버전

In [3]:
!java --version
!javac --version

java 11.0.12 2021-07-20 LTS
Java(TM) SE Runtime Environment 18.9 (build 11.0.12+8-LTS-237)
Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.12+8-LTS-237, mixed mode)
javac 11.0.12


- python 버전

In [4]:
!python --version

import sys
print(sys.version)

Python 3.8.11
3.8.11 (default, Aug  6 2021, 09:57:55) [MSC v.1916 64 bit (AMD64)]


- spark 실행 화면

```
(py3.8) C:\Users\Nunu>pyspark
Python 3.8.11 (default, Aug  6 2021, 09:57:55) [MSC v.1916 64 bit (AMD64)] :: Anaconda, Inc. on win32
Type "help", "copyright", "credits" or "license" for more information.
21/09/18 18:55:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.1.2
      /_/

Using Python version 3.8.11 (default, Aug  6 2021 09:57:55)
Spark context Web UI available at http://DESKTOP-7TC72N3:4040
Spark context available as 'sc' (master = local[*], app id = local-1631958957728).
SparkSession available as 'spark'.
>>>
```

---
## 문제 3: 
다음 파일을 읽어서 RDD를 생성하고, 5줄을 화면출력하세요.

결과가 깨져보인다면, 그 이유를 간단히 적으세요.

* 1) 경기도 의정부시 인구현황 (파일명: ```경기도 의정부시_인구현황_20200904```)

https://www.data.go.kr/data/15009613/fileData.do


* 2) 제주특별자치도 서귀포시 내 연도별 65세이상 인구수 및 고령화비율, 노령화지수 현황 (파일명: ```제주특별자치도 서귀포시_고령화비율및노령화지수현황_20200623```)

https://www.data.go.kr/data/15051545/fileData.do

- SparkSession 생성하기

In [5]:
import pyspark
import os

myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

### 1) 경기도 의정부시 인구현황 RDD 읽기

In [6]:
popRdd = spark.sparkContext\
    .textFile(os.path.join("data","경기도 의정부시_인구현황_20210910.csv"), use_unicode=True)

popRdd.take(5)

['�������,�α���(��),�α���(��),�α���(��),������(��),������(��),������(��),����,�����,������α�,���������,�����μ���,�μ���ȭ��ȣ,�����ͱ�������',
 '������1��,32292,16538,15754,6.97,3.57,3.4,104.98,19998,1.61,�����ν�,�ο����ǰ�,031-828-2466,2021-09-10',
 '������2��,31380,15608,15772,6.77,3.37,3.4,98.96,16410,1.91,�����ν�,�ο����ǰ�,031-828-2466,2021-09-10',
 'ȣ��1��,36124,17595,18529,7.8,3.8,4,94.96,15653,2.31,�����ν�,�ο����ǰ�,031-828-2466,2021-09-10',
 'ȣ��2��,34957,16923,18034,7.54,3.65,3.89,93.84,13683,2.55,�����ν�,�ο����ǰ�,031-828-2466,2021-09-10']

###  2) 제주특별자치도 서귀포시 내 연도별 65세이상 인구수 및 고령화비율, 노령화지수 현황 RDD 읽기

In [7]:
agedRdd = spark.sparkContext\
    .textFile(os.path.join("data", "제주특별자치도 서귀포시_고령화비율및노령화지수현황_20210831.csv"), use_unicode=True)

agedRdd.take(5)

['������,�������� �α���,65���̻� �α��� ,14������ �α���,���ȭ����,���ȭ����,�����ͱ�������',
 '2012,154057,25826,22861,16.76,112.97,2021-08-31',
 '2013,155641,26936,22393,17.31,120.29,2021-08-31',
 '2014,158512,27877,22058,17.59,126.38,2021-08-31',
 '2015,164519,28979,22362,17.61,129.59,2021-08-31']


- 결과가 깨져있는 이유:

    csv 파일을 다운로드 받을 때 한글이 깨지기 때문이다.
    
### 3) 한글 깨짐 해결 방법 (binaryFile)
그림과 같은 이진수로 이루어진 파일을 읽을 때 사용

In [10]:
popRddBin = spark.sparkContext.binaryFiles(os.path.join("data","경기도 의정부시_인구현황_20210910.csv"))

_my = popRddBin.map(lambda x : x[1].decode('euc-kr'))
_my.take(5)

['행정기관,인구수(계),인구수(남),인구수(여),구성비(계),구성비(남),구성비(여),성비,세대수,세대당인구,관리기관명,관리부서명,부서전화번호,데이터기준일자\r\n의정부1동,32292,16538,15754,6.97,3.57,3.4,104.98,19998,1.61,의정부시,민원여권과,031-828-2466,2021-09-10\r\n의정부2동,31380,15608,15772,6.77,3.37,3.4,98.96,16410,1.91,의정부시,민원여권과,031-828-2466,2021-09-10\r\n호원1동,36124,17595,18529,7.8,3.8,4,94.96,15653,2.31,의정부시,민원여권과,031-828-2466,2021-09-10\r\n호원2동,34957,16923,18034,7.54,3.65,3.89,93.84,13683,2.55,의정부시,민원여권과,031-828-2466,2021-09-10\r\n장암동,20314,9714,10600,4.38,2.1,2.29,91.64,8604,2.36,의정부시,민원여권과,031-828-2466,2021-09-10\r\n신곡1동,43159,21205,21954,9.31,4.58,4.74,96.59,17990,2.4,의정부시,민원여권과,031-828-2466,2021-09-10\r\n신곡2동,47852,23232,24620,10.33,5.01,5.31,94.36,19218,2.49,의정부시,민원여권과,031-828-2466,2021-09-10\r\n송산1동,42817,21276,21541,9.24,4.59,4.65,98.77,18811,2.28,의정부시,민원여권과,031-828-2466,2021-09-10\r\n송산2동,33565,16601,16964,7.24,3.58,3.66,97.86,13216,2.54,의정부시,민원여권과,031-828-2466,2021-09-10\r\n송산3동,46892,22772,24120,10.12,4.91,5.21,94.41,17926,2.62,의정부시,민원여권과,031-828-2466

In [13]:
popList = _my.map(lambda x: x.split()).take(3)
popList[0][0] # 헤더만 읽기

'행정기관,인구수(계),인구수(남),인구수(여),구성비(계),구성비(남),구성비(여),성비,세대수,세대당인구,관리기관명,관리부서명,부서전화번호,데이터기준일자'

#### 4. 데이터 분리하기 (DataFrame)

In [14]:
popDf = spark\
            .read.option("charset", "euc-kr")\
            .option("header", "true")\
            .csv(os.path.join("data","경기도 의정부시_인구현황_20210910.csv"))
popDf.show(5)

+---------+----------+----------+----------+----------+----------+----------+------+------+----------+----------+----------+------------+--------------+
| 행정기관|인구수(계)|인구수(남)|인구수(여)|구성비(계)|구성비(남)|구성비(여)|  성비|세대수|세대당인구|관리기관명|관리부서명|부서전화번호|데이터기준일자|
+---------+----------+----------+----------+----------+----------+----------+------+------+----------+----------+----------+------------+--------------+
|의정부1동|     32292|     16538|     15754|      6.97|      3.57|       3.4|104.98| 19998|      1.61|  의정부시|민원여권과|031-828-2466|    2021-09-10|
|의정부2동|     31380|     15608|     15772|      6.77|      3.37|       3.4| 98.96| 16410|      1.91|  의정부시|민원여권과|031-828-2466|    2021-09-10|
|  호원1동|     36124|     17595|     18529|       7.8|       3.8|         4| 94.96| 15653|      2.31|  의정부시|민원여권과|031-828-2466|    2021-09-10|
|  호원2동|     34957|     16923|     18034|      7.54|      3.65|      3.89| 93.84| 13683|      2.55|  의정부시|민원여권과|031-828-2466|    2021-09-10|
|   장암동|     20314|      9714|     10600